In [27]:
import numpy as np
from math import sqrt
from sklearn import preprocessing

In [28]:
data = np.loadtxt("data/spambase.data", dtype='f', delimiter=',')

print (data)
print (data.dtype)

max_value_in_data = np.zeros(57)

for lin in data:
    for x in range(0, 57):
        max_value_in_data[x] = max( max_value_in_data[x], lin[x] )
        

[[0.000e+00 6.400e-01 6.400e-01 ... 6.100e+01 2.780e+02 1.000e+00]
 [2.100e-01 2.800e-01 5.000e-01 ... 1.010e+02 1.028e+03 1.000e+00]
 [6.000e-02 0.000e+00 7.100e-01 ... 4.850e+02 2.259e+03 1.000e+00]
 ...
 [3.000e-01 0.000e+00 3.000e-01 ... 6.000e+00 1.180e+02 0.000e+00]
 [9.600e-01 0.000e+00 0.000e+00 ... 5.000e+00 7.800e+01 0.000e+00]
 [0.000e+00 0.000e+00 6.500e-01 ... 5.000e+00 4.000e+01 0.000e+00]]
float32


Isso mostra que teremos que tratar os dados antes de rodar __Online SVM via Online Gradient Descent__

Temos ao menos duas etapas:
1 - Transformar os valores de classe ( variável target ) de 0, 1 para -1 ,1
2 - Temos que fazer uma normalização dos parâmetros. Pois alguns dados são percentuais e estão no range [0, 100] e outros podem atingir valores maiores como por exemplo 15841. Não queremos que um atributo tenha uma importância maior que outro por ser representado em uma escala diferente. Vamos fazer uma normalização __hard__ que é mapear o maior valor de uma determinada feature para 1 e dividir os outros valores dessa feature específica pelo maior valor encontrado.

Se a normalização hard se mostrar ruim na prática, podemos pensar em outras normalizações. ( Soft usando mediana e desvio-padrão )..



In [30]:
# vamos alterar os valores da variável target de (0, 1) para (-1, 1)
for lin in data:
    if lin[57] == 0:
        lin[57] = -1
        
# Agora vamos fazer a normalizacao hard
normalized_data = data.copy()

print( normalized_data.dtype )

for lin in data:
    for x in range(0, 57):
        max_value_in_data[x] = max( max_value_in_data[x], lin[x] )
        
for lin in normalized_data:
    for x in range(0, 57):
        lin[x] = (lin[x] / max_value_in_data[x] )

        
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(data)
        

float32


In [31]:
x_scaled

array([[0.00000000e+00, 4.48179282e-02, 1.25490189e-01, ...,
        6.00720895e-03, 1.74873751e-02, 1.00000000e+00],
       [4.62555066e-02, 1.96078438e-02, 9.80392173e-02, ...,
        1.00120148e-02, 6.48358613e-02, 1.00000000e+00],
       [1.32158585e-02, 0.00000000e+00, 1.39215678e-01, ...,
        4.84581478e-02, 1.42550513e-01, 1.00000000e+00],
       ...,
       [6.60792962e-02, 0.00000000e+00, 5.88235334e-02, ...,
        5.00600727e-04, 7.38636404e-03, 0.00000000e+00],
       [2.11453736e-01, 0.00000000e+00, 0.00000000e+00, ...,
        4.00480581e-04, 4.86111175e-03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.27450973e-01, ...,
        4.00480581e-04, 2.46212143e-03, 0.00000000e+00]], dtype=float32)

In [32]:
normalized_data

array([[ 0.00000000e+00,  4.48179282e-02,  1.25490189e-01, ...,
         6.10671751e-03,  1.75493974e-02,  1.00000000e+00],
       [ 4.62555066e-02,  1.96078438e-02,  9.80392173e-02, ...,
         1.01111224e-02,  6.48948923e-02,  1.00000000e+00],
       [ 1.32158585e-02,  0.00000000e+00,  1.39215678e-01, ...,
         4.85534072e-02,  1.42604634e-01,  1.00000000e+00],
       ...,
       [ 6.60792962e-02,  0.00000000e+00,  5.88235334e-02, ...,
         6.00660744e-04,  7.44902482e-03, -1.00000000e+00],
       [ 2.11453736e-01,  0.00000000e+00,  0.00000000e+00, ...,
         5.00550610e-04,  4.92393179e-03, -1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.27450973e-01, ...,
         5.00550610e-04,  2.52509303e-03, -1.00000000e+00]], dtype=float32)

Breve descrição de Online SVM via Online Gradient Descent
* No instante t
    * Selecionamos um vetor pt
    * Recebemos uma nova instância do dataset (yt, zt), onde yt são os atributos e zt é a classe correspondente
    * Vamos tomar uma Hinge Loss definida como max(0, 1 - zt * <pt, yt> )
    * Utilizaremos o gradiente da função de perda do instante t para calcular o próximo p(t+1)
    
Vamos também utilizar sempre pt pertencente ao espaço euclideano de norma <= 1.

A princípio o p0 pode ser definido arbitrariamente, mas como vimos descrito em alguns lugares da literatura a inicialização com o vetor nulo vamos adotar essa estratégia.
   

In [4]:
# Aqui algumas funções auxiliares úteis
def hinge_loss( p_t, z_t, y_t):
    return max(0, 1 - np.dot(p_t, y_t) * z_t )

def project_into_euclidean_norm_1( p_t ):
    norm = np.linalg.norm(p_t)
    norm_p = p_t.copy()
    if norm <= 1:
        return norm_p
    else:
        norm_p = np.true_divide( norm_p, norm )
        return norm_p

def get_gradient( y_t, z_t ):
    gradient = y_t.copy()
    gradient = gradient * (-z_t)
    return gradient



In [5]:
'''
# Vamos armazenar os pt obtidos ao longo do tempo, para poder analisar quão rápido o método alcança bons classificadores
# E no final vamos testar os valores obtidos como classificadores para o dataset completo e analisar a acurácia
 all_p = []

current_p = [0] * 57

all_p.append( current_p )

# Temos algumas estratégias para definir o eta utilizado para atualizar p_t
# Vamos usar o eta proposto por Zinkevich que é da ordem de 1/sqrt(t)
# Que vai diminuindo conforme a quantidade de instancias processadas aumenta

# Se tivéssemos a segunda derivada da função de perda poderiamos usar 
# uma outra estratégia descoberta por Hazan em que eta_t = 1 / Ht, onde H é 
# um valor relacionado com a segunda derivada

# Essas estratégias para eta são definidas aqui (http://www.mit.edu/~rakhlin/papers/adaptive.pdf)


cumulative_loss = 0
# Agora vamos processar o dataset uma instancia de cada vez, num setup online

t = 1

for instance in normalized_data:
    instance_features = instance[0:57] # slices are semi-open intervals (:, [0, 57)
    target_class = instance[57]
    # vamos tomar a perda hinge relativo a current_p
    current_loss = hinge_loss( current_p, target_class, instance_features )
    cumulative_loss += current_loss
    
    if t % 100 == 0:
        print('perda na iteracao %d = %f' % (t, current_loss) )
    
    # Agora vamos usar o gradiente / subgradiente para fazer a atualizacao de p_(t + 1)
    # Se hinge_loss = 0, posso usar o subgradiente definido pela reta y = 0, e não mexer em p_t
    
    # O caso de fato interessante é quando a perda é positiva e queremos seguir no sentido contrário do subgradiente
    if current_loss != 0:
        gradient = get_gradient( instance_features, target_class )
        current_eta = 1 / sqrt(t)
        # p_t = p_(t - 1) - eta_t * ( gradiente da função de perda )
        for coord in range(57):
            current_p[coord] = (current_p[coord] - current_eta * gradient[coord])
        
        current_p = project_into_euclidean_norm_1( current_p )
            
        
    all_p.append( current_p ) # Adding current support vector to the list
    t += 1 # increment in one the number of processed instances
'''

"\n# Vamos armazenar os pt obtidos ao longo do tempo, para poder analisar quão rápido o método alcança bons classificadores\n# E no final vamos testar os valores obtidos como classificadores para o dataset completo e analisar a acurácia\n all_p = []\n\ncurrent_p = [0] * 57\n\nall_p.append( current_p )\n\n# Temos algumas estratégias para definir o eta utilizado para atualizar p_t\n# Vamos usar o eta proposto por Zinkevich que é da ordem de 1/sqrt(t)\n# Que vai diminuindo conforme a quantidade de instancias processadas aumenta\n\n# Se tivéssemos a segunda derivada da função de perda poderiamos usar \n# uma outra estratégia descoberta por Hazan em que eta_t = 1 / Ht, onde H é \n# um valor relacionado com a segunda derivada\n\n# Essas estratégias para eta são definidas aqui (http://www.mit.edu/~rakhlin/papers/adaptive.pdf)\n\n\ncumulative_loss = 0\n# Agora vamos processar o dataset uma instancia de cada vez, num setup online\n\nt = 1\n\nfor instance in normalized_data:\n    instance_featur

Vamos agora __encapsular__ o que foi implementado acima em um método, para poder testar diversos parâmetros de __eta__. E também vamos fazer uma função que mede a acurácia de um determinado vetor __p__ para classificar os dados.



In [6]:
def online_svm(data, eta_fnc):
    current_p = np.zeros(57)
    all_p = [ current_p ]
    cumulative_loss = 0
    # Agora vamos processar o dataset uma instancia de cada vez, num setup online
    t = 1

    for instance in data:
        instance_features = instance[0:57] # slices are semi-open intervals (:, [0, 57)
        target_class = instance[57]
        # vamos tomar a perda hinge relativo a current_p
        current_loss = hinge_loss( current_p, target_class, instance_features )
        cumulative_loss += current_loss
        # Agora vamos usar o gradiente / subgradiente para fazer a atualizacao de p_(t + 1)
        # Se hinge_loss = 0, posso usar o subgradiente definido pela reta y = 0, e não mexer em p_t

        # O caso de fato interessante é quando a perda é positiva e queremos seguir no sentido contrário do subgradiente
        if current_loss > 0:
            gradient = get_gradient( instance_features, target_class )
            current_eta = eta_fnc( t ) # Eta calculado em funcao de t, de acordo com a funcao recebida como parametro
            
            # p_t = p_(t - 1) - eta_t * ( gradiente da função de perda )
            cpy = current_p.copy()
            
            for coord in range(57):
                cpy[coord] = (current_p[coord] - current_eta * gradient[coord])

            cpy = project_into_euclidean_norm_1( cpy )
            current_p = cpy.copy()
            
        tmp = np.append(all_p, [cpy], axis = 0 ) # Adding current support vector to the list
        all_p = tmp.copy()
        
        t += 1 # increment in one the number of processed instances
    return all_p

Agora que encapsulamos o método em uma função que recebe como parâmetro uma função para o cálculo de eta, podemos começar a realizar alguns experimentos.

Antes disso, vou criar uma função que recebe um determinado vetor p, usa ele para __classificar__ todo o dataset e retorna a acurácia desse vetor.

In [7]:
def check_accuracy(data, classifier_vector):
    correct = 0
    total = 0
    for instance in data:
        instance_features = instance[0:57].copy()
        target_class = instance[57].copy()
        dot_prod = np.dot( instance_features, classifier_vector )
        predicted_class = -1
        if dot_prod >= 0:
            predicted_class = 1
        if target_class == predicted_class:
            correct += 1
        total += 1
    
    return ( correct / total )


def eta( t ):
    return 1 / sqrt(t)

def eta_constante( t ):
    return 0.05

test_data = normalized_data.copy()

results2 = online_svm( test_data, eta )
start_acc = check_accuracy( test_data, results2[0])
end_acc = check_accuracy( test_data, results2[-1])
print( start_acc, end_acc )

np.random.seed(13)
np.random.shuffle( test_data )


results = online_svm( test_data, eta )
start_acc2 = check_accuracy( test_data, results[0])
end_acc2 = check_accuracy( test_data, results[-1])
print( start_acc2, end_acc2 )



primeiro vetor =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.39404477287546186 0.6059552271245382
primeiro vetor =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.39404477287546186 0.780699847859161
